In [ ]:
test_string = "2333133121414131402"
test_string2 = "12345"
print(test_string2[::2])
print(test_string2[1::2])

135
24


In [ ]:
file_stack = list(map(lambda x : int(x), list(test_string2[::2])))
free_space_stack = list(map(lambda x : int(x), list(test_string2[1::2])))
file_iter = iter(file_stack)
free_iter = iter(free_space_stack)

id = 0
for file in file_iter:
    print(str(id)*file, end="")
    id += 1
    try:
        free_space = next(free_iter)
        print("."*free_space,end="")
    except:
        print()

0..111....22222


In [26]:
class FileNode():
    head = None
    tail = None
    next_file_id = 0
    def __init__(self, file_size, free_space):
        self.filestr = str(FileNode.next_file_id)*file_size
        FileNode.next_file_id += 1
        self.free_space = free_space
        self.next = None
        self.prev = None

        if not FileNode.head:
            FileNode.head = self
            FileNode.tail = self
        else:
            FileNode.tail.next = self
            self.prev = FileNode.tail
            FileNode.tail = self
    def __repr__(self):
        return self.filestr
    
    def pop(self):
         val = int(self.filestr[-1])
         self.filestr = self.filestr[:-1]
         self.free_space += 1
         return val
    
def file_free_lists(input_str):
    file_size_list = list(map(lambda x:int(x), list(input_str[::2])))
    free_space_stack = list(map(lambda x:int(x), list(input_str[1::2])))
    return (file_size_list, free_space_stack)
    
def process_disk_string(disk_str):
        file_sizes, free_spaces = file_free_lists(disk_str)
        if (len(file_sizes) < len(free_spaces)):
             raise("Number of Files is less than number of free spaces for some reason.")
        if (len(file_sizes) - len(free_spaces) == 1):
             free_spaces.append(0)
        pairs = zip(file_sizes, free_spaces)
        
        for pair in pairs:
             FileNode(pair[0], pair[1])
        return 0

def test_FileNode():
    test_string2 = "2333133121414131402"
    process_disk_string(test_string2)
    
    ptr = FileNode.head
    while ptr:
         print(str(ptr) + "."*ptr.free_space, end="")
         ptr = ptr.next
    print()
    ptr2 = FileNode.tail
    while ptr2:
         print(ptr2)
         ptr2 = ptr2.prev
    print("---")
    print(FileNode.tail)
    FileNode.tail.pop()
    print(FileNode.tail)

def print_filenodes():
    ptr = FileNode.head
    while ptr:
        print(str(ptr) + "."*ptr.free_space, end="")
        ptr = ptr.next
    print()

def get_disk_str():
    diskstr = ""
    ptr = FileNode.head
    while ptr:
        diskstr += ptr.filestr
        ptr = ptr.next
    return diskstr

"""Moves file blocks from tail_filenode to head_filenode. Returns head_filenode finished"""
def move_files_backwards(head_filenode, tail_filenode):
    if len(tail_filenode.filestr) == 0:
        print("end")
        return
    while len(tail_filenode.filestr) > 0:
        if head_filenode == tail_filenode:
            return tail_filenode
        if head_filenode.free_space <= 0:
            return move_files_backwards(head_filenode.next, tail_filenode)
        char = str(tail_filenode.pop())
        head_filenode.filestr += char
        head_filenode.free_space -= 1
    return head_filenode

def checksum(diskstr):
    sum = 0
    for i in range(len(diskstr)):
        sum += i * int(diskstr[i])
    return sum
def part1():
    easy_input = "12345"
    better_input = "2333133121414131402"
    file = open("input9.txt", "r")
    
    input = file.read().replace("\n","")
    
    print("IN:", input)
    process_disk_string(input)
    
    head = FileNode.head
    tail = FileNode.tail
    
    while head != tail.next:
        head = move_files_backwards(head, tail)
        tail = tail.prev
    
    print_filenodes()
    return

part1()

IN: ....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [48]:
class DiskNode():
    head = None
    tail = None
    next_file_id = 0

    def __init__(self, file_size, free_space):
        self.blocks = [DiskNode.next_file_id for i in range(file_size)]
        self.next_free = len(self.blocks)
        self.free_space = free_space
        self.file_size = file_size
        self.next = None
        self.prev = None
        DiskNode.next_file_id += 1

        free_blocks = ["." for j in range(free_space)]
        self.blocks += free_blocks

        if not DiskNode.head:
            DiskNode.head = self
            DiskNode.tail = self
        else:
            DiskNode.tail.next = self
            self.prev = DiskNode.tail
            DiskNode.tail = self
    
    def __repr__(self):
        return self.blocks
    
    def pop(self):
        val = self.blocks[self.next_free - 1]
        self.blocks[self.next_free - 1] = "."
        self.next_free = self.next_free - 1
        self.free_space += 1
        return val
    
    def insert(self, val):
        if self.free_space <= 0:
            return -1
        else:
            self.blocks[self.next_free] = val
            self.next_free += 1
            self.free_space -= 1
            return 0
        
    def reset():
        DiskNode.head = None
        DiskNode.tail = None
        DiskNode.next_file_id = 0

def file_free_lists(input_str):
    file_size_list = list(map(lambda x:int(x), list(input_str[::2])))
    free_space_stack = list(map(lambda x:int(x), list(input_str[1::2])))
    return (file_size_list, free_space_stack)

def process_disk_string(disk_str):
        if (DiskNode.head) or (DiskNode.tail) or (DiskNode.next_file_id != 0):
            DiskNode.reset()
        file_sizes, free_spaces = file_free_lists(disk_str)
        if (len(file_sizes) < len(free_spaces)):
             raise("Number of Files is less than number of free spaces for some reason.")
        if (len(file_sizes) - len(free_spaces) == 1):
             free_spaces.append(0)
        pairs = zip(file_sizes, free_spaces)
        
        for pair in pairs:
             DiskNode(pair[0], pair[1])
        return 0

def print_disk():
    ptr = DiskNode.head
    while ptr:
       for block in ptr.blocks:
           print(block,end="")
       ptr = ptr.next
    print()

def test_DiskNode():
    test_string2 = "2333133121414131402"
    process_disk_string(test_string2)
    ptr = DiskNode.head
    while ptr:
       for block in ptr.blocks:
           print(block,end="")
       ptr = ptr.next
    print()

    ptr2 = DiskNode.tail
    while ptr2:
        for block in ptr2.blocks:
            print(block,end="")
        print()
        ptr2 = ptr2.prev
    print("---")

def move_blocks_to_head(head_node, tail_node):
    while tail_node.next_free > 0:
        if head_node == tail_node:
            return tail_node
        if head_node.free_space <= 0:
            return move_blocks_to_head(head_node.next, tail_node)
        err = head_node.insert(val = tail_node.pop())
    return head_node

def calc_checksum():
    sum = 0
    i = 0
    head = DiskNode.head
    while head:
        for block in head.blocks:
            if type(block) != int:
                break
            sum += block*i
            i += 1
        head = head.next
    return sum


def part1():
    print("PART 1:")
    easy_input = "2333133121414131402"
    file = open("input9.txt", "r")
    input = file.read().replace("\n", "")

    process_disk_string(input)
    head = DiskNode.head
    tail = DiskNode.tail

    while head != tail.next:
        head = move_blocks_to_head(head, tail)
        tail = tail.prev
    print_disk()
    print(calc_checksum())
    return

# Attempts to move all of node's file blocks to a contiguous free block.
# Returns -1 if the operation failed, 0 upon success.
def whole_file_compactor(head, node):
    while head.free_space < node.file_size:
        head = head.next
        if head == node:
            return head
    
    for i in range(node.file_size):
        head.insert(node.pop())
    return 0
part1()

PART 1:
................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [127]:
class CompactBlock():
    next_file_id = 0
    head_free = None
    tail_free = None
    tail_file = None
    true_head = None
    true_tail = None
    def __init__(self, blocksize, is_free):
        self.true_next = None
        if not CompactBlock.true_head:
            CompactBlock.true_head = self
            CompactBlock.true_tail = self
        else:
            CompactBlock.true_tail.true_next = self
            CompactBlock.true_tail = self
        
        self.blocksize = blocksize
        self.is_free = is_free
        self.blocks = []
        self.last_index = 0
        self.next = None
        self.prev = None
        
        if not is_free:
            self.blocks = [CompactBlock.next_file_id for i in range(blocksize)]
            CompactBlock.next_file_id += 1
            self.last_index = blocksize - 1
            if not CompactBlock.tail_file:
                CompactBlock.tail_file = self
            else:
                CompactBlock.tail_file.next = self
                self.prev = CompactBlock.tail_file
                CompactBlock.tail_file = self
        else:
            if not CompactBlock.head_free:
                CompactBlock.head_free = self
                CompactBlock.tail_free = self
            else:
                CompactBlock.tail_free.next = self
                self.prev = CompactBlock.tail_free
                CompactBlock.tail_free = self
    
    def __repr__(self):
        string = ""
        for val in self.blocks:
            string += str(val)
        if self.is_free:
            return string + "."*self.blocksize
        else:
            return string
        
    def pop(self):
        val = self.blocks[self.last_index]
        self.blocks[self.last_index] = "."
        self.last_index -= 1
        return val
    
    def insert(self, val):
        if self.blocksize <= 0:
            return -1
        else:
            self.blocks.append(val)
            self.blocksize -= 1
            return 0

def print_compact_disk():
    node = CompactBlock.true_head
    while node:
        print(node,end="")
        node = node.true_next
    print()
    return

def generate_block_list():
    node = CompactBlock.true_head
    blocks = []
    while node:
        if not node.is_free:
            blocks += node.blocks
        else:
            blocks += node.blocks + ["." for i in range(node.blocksize)]
        node = node.true_next
    return blocks

def test_Compact():
    CompactBlock(3, False)
    CompactBlock(3, True)
    CompactBlock(4, False)
    CompactBlock(1, True)
    print(CompactBlock.tail_file)
    CompactBlock(2, False)
    CompactBlock(10, True)
    print(CompactBlock.head_free)
    print(CompactBlock.tail_free)
    print_compact_disk()
    return

def file_free_lists(input_str):
    file_size_list = list(map(lambda x:int(x), list(input_str[::2])))
    free_space_stack = list(map(lambda x:int(x), list(input_str[1::2])))
    return (file_size_list, free_space_stack)

def read_disk(diskstr):
    file_sizes, free_spaces = file_free_lists(diskstr)
    if (len(file_sizes) < len(free_spaces)):
        raise("Number of Files is less than number of free spaces for some reason.")
    if (len(file_sizes) - len(free_spaces) == 1):
        free_spaces.append(0)
    pairs = zip(file_sizes, free_spaces)
    for pair in pairs:
        CompactBlock(pair[0], False)
        CompactBlock(pair[1], True)
    return

def move_into_free(free_node, file):
    if free_node.blocksize < file.blocksize:
        raise "Warning: Free node does not have enough space!"
    
    for i in range(file.blocksize):
        free_node.insert(file.pop())
    return

def attempt_to_insert(head, file):
    if not head.is_free:
        raise "Warning: attempting to insert into a non-free block!"
    while head != file and head:
        if (not head.is_free) or (head.blocksize < file.blocksize):
            head = head.true_next
        else:
            move_into_free(head, file)
            return
    
def calc_checksum():
    blocklist = generate_block_list()
    sum = 0
    i=0
    for block in blocklist:
        if type(block) == int:
            sum += block*i
        i += 1
    return sum

def part2():
    easy_input = "12345"
    test_input = "233313312141413140253"
    test_input2 = "2333133121414131402"
    file = open("input9.txt", "r")

    #input = test_input2
    input = file.read().replace("\n", "")
    read_disk(input)
    print_compact_disk()
    file = CompactBlock.tail_file
    while file:
        attempt_to_insert(CompactBlock.head_free, file)
        file = file.prev
    print(calc_checksum())
    
    
    return
part2()

0.......1.222222......3333333...444444...556....77777777.........88888.999999......101010101010101010..111111....12121212121212....1313131313131313.....14.........1515..16.......171717171717...1818181818...191919191919.....202020....212121212121212121....222222....23232323...242424242425252525....26262626........2727272727272727.........282828.2929292929......3030303030303030303131313131313131....32323232........333333333334...35353535353535.....363636........37373737.....3838383838.393939404040404040.41414141414141.......42....4343434444444445454545454545..4646464646..4747........484848.......49494949494949.........505050...51.........525252525252525252........53........54545454..5555565656........57575757575757....5858585858........5959........6060606060.....616161616161...6262626262......636363636363....64646464646464......6565656565656565..666666666666666666.......67676767676767........68686868....6969..707070707070707070.....7171...727272727272......737373737373737373......7474747